<a href="https://colab.research.google.com/github/simabest/Simon/blob/master/Project_Generator_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

# Define the project structure and content
project_name = "dashen-lottery"
folders = [
    project_name,
    f"{project_name}/src",
    f"{project_name}/public"
]

files = {
    f"{project_name}/package.json": """{
  "name": "dashen-bank-lottery",
  "private": true,
  "version": "1.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "lucide-react": "^0.284.0",
    "xlsx": "https://cdn.sheetjs.com/xlsx-0.20.1/xlsx-0.20.1.tgz"
  },
  "devDependencies": {
    "@vitejs/plugin-react": "^4.0.3",
    "autoprefixer": "^10.4.14",
    "postcss": "^8.4.27",
    "tailwindcss": "^3.3.3",
    "vite": "^4.4.5"
  }
}""",
    f"{project_name}/index.html": """<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Dashen Bank Lottery</title>
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.jsx"></script>
  </body>
</html>""",
    f"{project_name}/vite.config.js": "import { defineConfig } from 'vite';\nimport react from '@vitejs/plugin-react';\n\nexport default defineConfig({\n  plugins: [react()],\n});",
    f"{project_name}/tailwind.config.js": "/** @type {import('tailwindcss').Config} */\nexport default {\n  content: ['./index.html', './src/**/*.{js,ts,jsx,tsx}'],\n  theme: { extend: {} },\n  plugins: [],\n};",
    f"{project_name}/postcss.config.js": "export default {\n  plugins: {\n    tailwindcss: {},\n    autoprefixer: {},\n  },\n};",
    f"{project_name}/src/main.jsx": "import React from 'react';\nimport ReactDOM from 'react-dom/client';\nimport App from './App.jsx';\nimport './index.css';\n\nReactDOM.createRoot(document.getElementById('root')).render(\n  <React.StrictMode>\n    <App />\n  </React.StrictMode>,\n);",
    f"{project_name}/src/index.css": "@tailwind base;\n@tailwind components;\n@tailwind utilities;",
    f"{project_name}/src/App.jsx": """import React, { useState, useEffect } from 'react';
import { Trophy, Download, Upload, Loader2, Building2, AlertCircle, Settings } from 'lucide-react';

export default function App() {
  const [employees, setEmployees] = useState([]);
  const [winners, setWinners] = useState([]);
  const [isDrawing, setIsDrawing] = useState(false);
  const [isUploading, setIsUploading] = useState(false);
  const [timer, setTimer] = useState(0);
  const [inputBudget, setInputBudget] = useState(30000000);

  useEffect(() => {
    const script = document.createElement('script');
    script.src = "https://cdn.sheetjs.com/xlsx-0.20.1/package/dist/xlsx.full.min.js";
    script.async = true;
    document.body.appendChild(script);
  }, []);

  useEffect(() => {
    let interval;
    if (isDrawing && timer > 0) {
      interval = setInterval(() => setTimer((prev) => prev - 1), 1000);
    } else if (timer === 0 && isDrawing) {
      finalizeBudgetDraw();
    }
    return () => clearInterval(interval);
  }, [isDrawing, timer]);

  const handleFileUpload = (e) => {
    const file = e.target.files[0];
    if (!file) return;
    setIsUploading(true);
    const reader = new FileReader();
    reader.onload = (evt) => {
      try {
        const wb = window.XLSX.read(evt.target.result, { type: 'binary' });
        const data = window.XLSX.utils.sheet_to_json(wb.Sheets[wb.SheetNames[0]]);
        setEmployees(data.map(item => ({
          employeeId: item['Employee ID'] || 'N/A',
          fullName: item['Full Name'] || 'Unknown',
          loanType: item['Loan Type'] || 'House',
          loanLimit: parseFloat(item['Loan Limit'] || 0)
        })));
      } catch (err) {
        alert("Excel Error: Ensure columns: Employee ID, Full Name, Loan Limit.");
      } finally {
        setIsUploading(false);
      }
    };
    reader.readAsBinaryString(file);
  };

  const startDraw = () => {
    if (employees.length === 0) return alert("Upload employee data first.");
    setWinners([]);
    setIsDrawing(true);
    setTimer(20);
  };

  const finalizeBudgetDraw = () => {
    const pool = [...employees].sort(() => Math.random() - 0.5);
    let currentTotal = 0;
    const selectedWinners = [];
    for (const emp of pool) {
      if (currentTotal + emp.loanLimit <= inputBudget) {
        selectedWinners.push(emp);
        currentTotal += emp.loanLimit;
      }
      if (currentTotal >= inputBudget) break;
    }
    setWinners(selectedWinners);
    setIsDrawing(false);
  };

  return (
    <div className="min-h-screen bg-slate-50 text-slate-900">
      <nav className="bg-[#003366] text-white p-4 flex justify-between items-center border-b-4 border-[#FFCC00]">
        <div className="flex items-center gap-2"><Building2 /><h1 className="text-xl font-bold">DASHEN BANK</h1></div>
      </nav>
      <main className="max-w-6xl mx-auto p-6 space-y-6">
        <div className="grid grid-cols-1 md:grid-cols-3 gap-4">
          <div className="bg-white p-4 rounded-xl shadow border">
            <p className="text-xs font-bold text-slate-500 uppercase">Budget (ETB)</p>
            <input type="number" value={inputBudget} onChange={(e) => setInputBudget(Number(e.target.value))} className="text-xl font-black w-full border-b outline-none" />
          </div>
          <div className="bg-white p-4 rounded-xl shadow border">
            <p className="text-xs font-bold text-slate-500 uppercase">Utilized</p>
            <p className="text-xl font-black text-emerald-600">{winners.reduce((s, w) => s + w.loanLimit, 0).toLocaleString()} ETB</p>
          </div>
          <label className="bg-blue-800 text-white p-4 rounded-xl shadow flex items-center justify-center font-bold cursor-pointer">
            {isUploading ? "Reading..." : "Upload Excel"}
            <input type="file" hidden onChange={handleFileUpload} accept=".xlsx, .xls" />
          </label>
        </div>
        <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
          <div className="bg-white rounded-2xl shadow h-[500px] flex flex-col border overflow-hidden">
            <div className="p-4 bg-slate-50 border-b font-bold">Pool ({employees.length})</div>
            <div className="flex-1 overflow-y-auto p-4">
              {employees.map((emp, i) => (
                <div key={i} className="flex justify-between border-b py-2 text-sm">
                  <span>{emp.fullName}</span><span className="font-bold">{emp.loanLimit.toLocaleString()}</span>
                </div>
              ))}
            </div>
            <button onClick={startDraw} disabled={isDrawing || employees.length === 0} className="m-4 p-4 bg-blue-800 text-white rounded-xl font-bold disabled:bg-slate-300">Run Draw</button>
          </div>
          <div className="bg-white rounded-2xl shadow h-[500px] flex flex-col border-2 border-yellow-400 overflow-hidden relative">
            <div className="p-4 bg-yellow-50 border-b font-bold">Winners</div>
            <div className="flex-1 overflow-y-auto p-4">
              {isDrawing && <div className="absolute inset-0 bg-white/90 z-10 flex items-center justify-center text-2xl font-black">{timer}s</div>}
              {winners.map((win, i) => (
                <div key={i} className="flex justify-between p-3 bg-yellow-50 mb-2 rounded border">
                  <span>{win.fullName}</span><span className="font-bold">{win.loanLimit.toLocaleString()}</span>
                </div>
              ))}
            </div>
          </div>
        </div>
      </main>
    </div>
  );
}"""
}

# Create folders
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Create files
for path, content in files.items():
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)

print(f"Project '{project_name}' created successfully!")

Project 'dashen-lottery' created successfully!
